In [1]:
import os, sys
# make sure pyspark tells workers to use python2 not 3 if both are
#installed

os.environ['PYSPARK_PYTHON'] = '/usr/bin/python2'
reload(sys)
sys.setdefaultencoding('utf-8')

import pyspark
from pyspark.sql import SQLContext

sc =  pyspark.SparkContext('local[*]')
sqlCtx = SQLContext(sc)

In [14]:
import csv

def unicode_csv_reader(unicode_csv_data, dialect=csv.excel, **kwargs):
    # csv.py doesn't do Unicode; encode temporarily as UTF-8:
    csv_reader = csv.reader(utf_8_encoder(unicode_csv_data),
                            dialect=dialect, **kwargs)
    for row in csv_reader:
        # decode UTF-8 back to Unicode, cell by cell:
        yield [unicode(cell, 'utf-8') for cell in row]

def utf_8_encoder(unicode_csv_data):
    for line in unicode_csv_data:
        yield line.encode('utf-8')

In [76]:
lines = sc.textFile("data/2000s_authors.csv")

In [81]:
header = lines.first()
rows = lines.filter(lambda line: line != header)
authors = rows.map(lambda line: list(unicode_csv_reader(line))).filter(lambda x: len(x) == 3).flatMap(lambda x: x[2])

In [82]:
import itertools   
coauthors = authors.flatMap(lambda a: [[tuple(sorted(coauth)), 1] for coauth in itertools.combinations(a.split('|'), 2)])

In [83]:
coauthors.reduceByKey(lambda x, y: x + y).take(20)

[((u'Iriarte Navarro, L.', u'Mart\xednez Rodr\xedguez, A.'), 1),
 ((u'Economic Research Service.', u'Liefert, William.'), 5),
 ((u'MacDonald, G. M.', u'Sheng, Y.'), 1),
 ((u'Smekens, Koen', u'van der Zwaan, Bob'), 1),
 ((u'Bacon, Nicholas J.', u'Farese, James P.'), 1),
 ((u'Tanaka, K.', u'Tsuchizaki, N.'), 1),
 ((u'Li, Lin', u'Li, Qianqian'), 1),
 ((u'Mentis, Andreas F.', u'Panayiotou, Joanna'), 1),
 ((u'Licciulli, F.', u'Liuni, S.'), 1),
 ((u'Kennett, JP', u'Lea, DW'), 1),
 ((u'Balla, Branko', u'Mocak, Jan'), 1),
 ((u'Fukumori, Akio', u'Haug-Kr\xf6per, Martina'), 1),
 ((u'Takami, Ikuo', u'Yoshimizu, Mamoru'), 1),
 ((u'Song, X.', u'Wang, J.'), 1),
 ((u'Bowie, Mike H.', u'Vink, Cor J.'), 1),
 ((u'Imaizumi, K.', u'Iwamoto, M.'), 3),
 ((u'Jursova, Michaela', u'Stopka, Pavel'), 1),
 ((u'Garcia, M.E.', u'Johnson, R.S.'), 2),
 ((u'Mitchell, Kendra', u'Reysenbach, Anna-Louise'), 1),
 ((u'Fab\xedola Villa', u'Fab\xedola Villa'), 1)]

In [80]:
authors = sc.parallelize([['a', 'b', 'c'], ['c', 'b']])
authors.flatMap(lambda a: [[tuple(sorted(coauth)), 1] for coauth in itertools.combinations(a, 2)]).collect()

[[('a', 'b'), 1], [('a', 'c'), 1], [('b', 'c'), 1], [('b', 'c'), 1]]